## 1.固定长度文本切分

按照固定的文本长度切分文本，不同分块之间可以有固定长度的重叠内容。

In [1]:
def fixed_length_split(text: str, chunk_size: int, chunk_overlap: int = 0) -> list:
    """Split text into fixed length chunks with optional overlap."""
    chunks = []
    for i in range(0, len(text), chunk_size - chunk_overlap):
        chunks.append(text[i:i + chunk_size])
    return chunks

# 测试数据
text = """文本分块（Text Chunking / Splitting），顾名思义，就是将原始的、可能非常庞大的文本资料\
（例如，一篇长篇报告、一本电子书、一个复杂的网页或者大量的 API 文档）分割成一系列更小、更易于处理的文本片段（Chunks）的过程。\
这些 Chunks 是 RAG 系统中信息处理的基本单元，它们将被送入 Embedding 模型进行向量化，然后存入向量数据库进行索引，最终服务于检索环节。"""
chunk_size, chunk_overlap = 50, 10

# 测试
chunks = fixed_length_split(text, chunk_size, chunk_overlap)
for idx, chunk in enumerate(chunks):
    print('-'*30 + f' Chunk {idx} ' + '-'*30)
    print(chunk)

------------------------------ Chunk 0 ------------------------------
文本分块（Text Chunking / Splitting），顾名思义，就是将原始的、可能非常庞大
------------------------------ Chunk 1 ------------------------------
原始的、可能非常庞大的文本资料（例如，一篇长篇报告、一本电子书、一个复杂的网页或者大量的 API 文
------------------------------ Chunk 2 ------------------------------
者大量的 API 文档）分割成一系列更小、更易于处理的文本片段（Chunks）的过程。这些 Chun
------------------------------ Chunk 3 ------------------------------
过程。这些 Chunks 是 RAG 系统中信息处理的基本单元，它们将被送入 Embedding 模
------------------------------ Chunk 4 ------------------------------
mbedding 模型进行向量化，然后存入向量数据库进行索引，最终服务于检索环节。
------------------------------ Chunk 5 ------------------------------
。


## 2.Markdown标题切分
根据 Markdown 原生标题切分文本内容，将相同标题级别的文本片段切分在同一个 chunk 中。

In [54]:
import re

def markdown_heading_split(text):
    chunks = []
    current_chunk = ''
    current_heading: str = None  # None, '#', '##', '###', etc.

    lines = text.splitlines()
    for line in lines:
        match = re.match(r'^(#+)\s*(.*)$', line)
        if match:
            heading, heading_text = match.groups()
            if current_heading is None:
                current_heading = heading
            if heading == current_heading:
                current_chunk += heading_text + '\n'
            else:
                chunks.append(current_chunk)
                current_chunk = heading_text + '\n'
                current_heading = heading
        else:
            current_chunk += line + '\n'
    if current_chunk:
        chunks.append(current_chunk)
    return chunks

# 测试数据
text = """
# Main Header

## Section 1
Content for section 1

### Subsection 1.1
Subsection content

## Section 2
Another section

## Section 3
test
"""

# 测试
chunks = markdown_heading_split(text)
for idx, chunk in enumerate(chunks):
    print('-'*30 + f' Chunk {idx} ' + '-'*30)
    print(chunk)

------------------------------ Chunk 0 ------------------------------

Main Header


------------------------------ Chunk 1 ------------------------------
Section 1
Content for section 1


------------------------------ Chunk 2 ------------------------------
Subsection 1.1
Subsection content


------------------------------ Chunk 3 ------------------------------
Section 2
Another section

Section 3
test



## 3. 递归文本切分
按照字符递归切分，递归分块使用一组分隔符以分层和迭代的方式将输入文本分成更小的块

Q: 为什么使用了 `RecursiveCharacterTextSplitter`，但是指定的 `chunk_overlap` 没有生效？  
A：因为 `RecursiveCharacterTextSplitter` 会按照字符列表的优先级递归切分，当不能再切分下去并且 `chunk` 长度还是大于 `chunk_size` 时，才会使用 `chunk_overlap`


In [68]:
def recursive_character_text_splitter(text, chunk_size, chunk_overlap=0, separators=["\n\n", "\n", "。"]):
    """
    手动实现 RecursiveCharacterTextSplitter 的功能
    :param text: 要分割的文本
    :param chunk_size: 每个文本块的最大长度
    :param chunk_overlap: 相邻文本块的重叠长度
    :param separators: 尝试分割文本的分隔符列表
    :return: 分割后的文本块列表
    """
    final_chunks = []
    if not text:
        return final_chunks
    # 尝试按分隔符分割文本
    separator = separators[0]
    if separator:
        splits = text.split(separator)
    else:
        splits = [text]
    # 检查每个分割后的部分
    good_splits = []
    for split in splits:
        # 分割后的文本长度小于 chunk_size，直接添加到 good_splits
        # 如果分割后的文本长度大于 chunk_size，递归调用，使用下一个分隔符继续分割，知道分割后的文本长度小于 chunk_size 或者没有更多分隔符
        # 最终 good_splits 中的每个元素都是长度小于 chunk_size 的文本
        if len(split) < chunk_size:
            good_splits.append(split)
        else:
            if len(separators) > 1:
                # 递归调用，使用下一个分隔符
                child_splits = recursive_character_text_splitter(split, chunk_size, chunk_overlap, separators[1:])
                good_splits.extend(child_splits)
            else:
                # 如果没有更多分隔符，直接分割
                for i in range(0, len(split), chunk_size - chunk_overlap):
                    good_splits.append(split[i:i + chunk_size])
    
    # 合并相邻的分割部分，保证每个 chunk 的长度都不超过 chunk_size
    for split in good_splits:
        if not final_chunks:
            final_chunks.append(split)
        else:
            last_chunk = final_chunks[-1]
            if len(last_chunk) + len(split) <= chunk_size:
                final_chunks[-1] = last_chunk + separator + split
            else:
                final_chunks.append(split)
    return final_chunks


# 测试数据
text = """文本分块（Text Chunking / Splitting）是什么？顾名思义，就是将原始的、可能非常庞大的文本资料\
（例如，一篇长篇报告、一本电子书、一个复杂的网页或者大量的 API 文档）分割成一系列更小、更易于处理的文本片段（Chunks）的过程！\
这些 Chunks 是 RAG 系统中信息处理的基本单元。它们将被送入 Embedding 模型进行向量化，然后存入向量数据库进行索引，最终服务于检索环节。
这是一个新段落测试文本。哈哈哈！"""
separators = ['\n', '。', '？']

# 测试
# chunks = recursive_split(text, delimeter)
chunks = recursive_character_text_splitter(text, 60, 15, separators)
for idx, chunk in enumerate(chunks):
    print('-'*30 + f' Chunk {idx} ' + '-'*30)
    print(chunk)

# 测试 RecursiveCharacterTextSplitter
# from langchain_text_splitters import RecursiveCharacterTextSplitter
# splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
# chunks = splitter.split_text(text)
# for idx, chunk in enumerate(chunks):
#     print('-'*30 + f' Chunk {idx} ' + '-'*30)
#     print(chunk)

------------------------------ Chunk 0 ------------------------------
文本分块（Text Chunking / Splitting）是什么？
------------------------------ Chunk 1 ------------------------------
顾名思义，就是将原始的、可能非常庞大的文本资料（例如，一篇长篇报告、一本电子书、一个复杂的网页或者大量的 API 文档）
------------------------------ Chunk 2 ------------------------------
网页或者大量的 API 文档）分割成一系列更小、更易于处理的文本片段（Chunks）的过程！这些 Chunks 是 RA
------------------------------ Chunk 3 ------------------------------
！这些 Chunks 是 RAG 系统中信息处理的基本单元。？
------------------------------ Chunk 4 ------------------------------
它们将被送入 Embedding 模型进行向量化，然后存入向量数据库进行索引，最终服务于检索环节。。
。
------------------------------ Chunk 5 ------------------------------
这是一个新段落测试文本。哈哈哈！



## 4. 根据语义切分

TODO: